In [3]:
import pandas as pd
import numpy as np
import datetime as dt

# Prepare the dataset

To prepare for the buyer features, we have to use the log dataset, which contains the behaviours of around 5000 frequent usersover 6 months.

In [49]:
df = pd.read_csv('product_log.csv')

# train test split

In [50]:
# ignore 2013-09, which serves as outcomes in test dataset
df['vmonth'] = df.vtime.str[: 7]

In [6]:
df.loc[:, 'vtime'] = df.vtime.apply(pd.to_datetime)

In [51]:
# keep 2013-08 as outcome, and use data before that to create features
train = df[df.vmonth != '2013-09'].reset_index(drop=True)
out = df[df.vmonth == '2013-09'].reset_index(drop=True)

# seller to user data

In [8]:
%store -r user_id
user_id = list(user_id)
seller_id = list(set(df.seller_id))

In [9]:
# filter training data to consider only the buyer who will purchase any item next month
train = train[train.user_id.isin(user_id)]
train = train.reset_index(drop=True) 

In [10]:
# create index based on the first state prediction
index = pd.MultiIndex.from_product([user_id, seller_id], names=['user_id', 'seller_id'])
b_s = pd.DataFrame(index=index)

## historical

In [11]:
# group training dataset by user_id, seller_id, action, for historical features
group = train.groupby(['user_id', 'seller_id', 'action']).count()
group = group.reset_index()

In [12]:
# purchase
to_merge = group[group.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'purchase'})

In [13]:
# click
to_merge = group[group.action == 'click'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'click'})

In [14]:
# cart
to_merge = group[group.action == 'cart'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'cart'})

In [15]:
b_s['click_buy_ratio'] = b_s.purchase / b_s.click
b_s['cart_buy_ratio'] = b_s.purchase / b_s.cart

### last month

In [16]:
# group training dataset by user_id, seller_id, action, but only consider July
last_month = train[train.vmonth == '2013-08'].groupby(['user_id', 'seller_id', 'action']).count()[['item_id']]
last_month = last_month.reset_index()

In [17]:
to_merge = last_month[last_month.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'purchase_last_m'})

In [18]:
to_merge = last_month[last_month.action == 'click'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'click_last_m'})

In [19]:
to_merge = last_month[last_month.action == 'cart'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'cart_last_m'})

In [20]:
b_s['click_buy_ratio_last_m'] = b_s.purchase_last_m / b_s.click_last_m
b_s['cart_buy_ratio_last_m'] = b_s.purchase_last_m / b_s.cart_last_m

### recent

In [21]:
recent = train[train.vtime > dt.datetime(2013, 8, 20)].groupby(['user_id', 'seller_id', 'action']).count()[['item_id']].reset_index()

In [22]:
to_merge = recent[recent.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'purchase_recent'})

In [23]:
to_merge = recent[recent.action == 'click'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'click_recent'})

In [24]:
to_merge = recent[recent.action == 'cart'][['user_id', 'seller_id', 'item_id']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'cart_recent'})

In [25]:
b_s['click_buy_ratio_recent'] = b_s.purchase_recent / b_s.click_recent
b_s['cart_buy_ratio_recent'] = b_s.purchase_recent / b_s.cart_recent

In [26]:
b_s= b_s.fillna(0)

### last

In [27]:
to_merge = train[train.action == 'alipay'].groupby(['user_id', 'seller_id']).max()[['vtime']]
to_merge['last_purchase'] = dt.datetime(2013, 9, 1) - to_merge.vtime
to_merge['last_purchase'] = to_merge.last_purchase.apply(lambda x: x.days)
to_merge = to_merge.reset_index()
b_s = b_s.merge(to_merge[['user_id', 'seller_id', 'last_purchase']], how='left', on=['user_id', 'seller_id'])

In [29]:
to_merge = train[train.action == 'click'].groupby(['user_id', 'seller_id']).max()[['vtime']]
to_merge['last_click'] = dt.datetime(2013, 9, 1) - to_merge.vtime
to_merge['last_click'] = to_merge.last_click.apply(lambda x: x.days)
to_merge = to_merge.reset_index()
b_s = b_s.merge(to_merge[['user_id', 'seller_id', 'last_click']], how='left', on=['user_id', 'seller_id'])

In [30]:
to_merge = train[train.action == 'cart'].groupby(['user_id', 'seller_id']).max()[['vtime']]
to_merge['last_cart'] = dt.datetime(2013, 9, 1) - to_merge.vtime
to_merge['last_cart'] = to_merge.last_cart.apply(lambda x: x.days)
to_merge = to_merge.reset_index()
b_s = b_s.merge(to_merge[['user_id', 'seller_id', 'last_cart']], how='left', on=['user_id', 'seller_id'])

In [31]:
b_s['last_purchase'] = b_s.last_purchase.fillna(121)
b_s['last_click'] = b_s.last_click.fillna(121)
b_s['last_cart'] = b_s.last_cart.fillna(121)

### sentiment

In [47]:
sentiment = pd.read_csv('reviewscore.csv')
to_merge = sentiment[['user_id', 'seller_id', 'sentimentscore_pos', 'sentimentscore_neu', 'sentimentscore_neg']]
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s[['sentimentscore_pos', 'sentimentscore_neu', 'sentimentscore_neg']] = b_s[['sentimentscore_pos', 'sentimentscore_neu', 'sentimentscore_neg']].fillna(0)

### outcome

In [52]:
g = out.groupby(['user_id', 'seller_id', 'action']).count()
g = g.reset_index()
to_merge = g[g.action == 'alipay'][['user_id', 'seller_id', 'item_id']]
to_merge['item_id'] = np.ones(len(to_merge))

In [53]:
b_s = b_s.merge(to_merge, how='left', on=['user_id', 'seller_id'])
b_s = b_s.rename(columns={'item_id': 'outcome'})
b_s['outcome'] = b_s.outcome.fillna(0)

In [55]:
b_s.to_csv('b_s_test.csv', index=False)